In [1]:
import config
import cx_Oracle
import re

pattern = re.compile(r'\s+')

In [41]:
# connect to Oracle DB
connection = None
try:
    connection = cx_Oracle.connect(
        config.username,
        config.password,
        config.dsn,
        encoding = "UTF-8"
    )
    print("Connected")
    print(connection.version)
except cx_Oracle.Error as error:
    print(error)

cursor = connection.cursor()

Connected
12.2.0.1.0


In [3]:
sqlfilter = "SELECT TABLE_NAME FROM ALL_TABLES WHERE TABLE_NAME LIKE '%STYLE%' AND TABLE_NAME LIKE '%G3E%'"
cursor.execute(sqlfilter)
fetch = cursor.fetchall()
tablelist = []
for table in fetch:
    tablelist.append(table[0])

In [4]:
print(tablelist)

['G3E_STYLETYPE_OPTLANG', 'G3E_STYLEOBJECTS_OPTLANG', 'G3E_STYLEFILTERS_OPTLANG', 'G3E_STYLECOLORS_OPTABLE', 'G3E_TEXTSTYLE_10100901', 'G3E_STYLERULE_0009', 'G3E_TEXTSTYLEFONT_OPTABLE', 'G3E_TEXTSTYLE', 'G3E_STYLE_0009', 'G3E_STYLETYPE_0009', 'G3E_STYLETYPE', 'G3E_STYLERULE', 'G3E_STYLEMAPPING_OPTABLE', 'G3E_STYLEMAPPING', 'G3E_STYLE', 'G3E_POINTSTYLEFONT_OPTABLE', 'G3E_POINTSTYLE', 'G3E_LINESTYLE', 'G3E_DIMSTYLEGROUP_OPTABLE', 'G3E_COMPOSITELINESTYLE', 'G3E_AREASTYLE', 'G3E_STYLELANGMAPPING', 'G3E_DIMSTYLEGROUP']


In [46]:
view_name = "V_ADM_GPB_SHP_1"
sql = "SELECT distinct(G3E_FNO) FROM {view_name}".format(view_name = view_name)
cursor.execute(sql)
FNO = cursor.fetchall()[0][0]
sql = "Select b.g3e_table, b.g3e_geometrytype from G3E_FEATURECOMPONENT a join G3E_COMPONENT b on a.G3E_CNO = b.G3E_CNO where a.G3E_FNO = {FNO}".format(FNO =FNO)
cursor.execute(sql)


140


In [6]:
sql = "SELECT TEXT FROM USER_VIEWS WHERE VIEW_NAME = '{view_name}'".format(view_name = view_name)
cursor.execute(sql)
fetch = cursor.fetchall()

In [7]:
txts = fetch[0][0]
txts = txts.rpartition("CASE")[2]
txts = txts.rpartition("ELSE")[0]
txts = re.sub(pattern,'',txts)
txts = txts.split("WHEN")

print(txts)

['', "TRIM(F.DISTRICT_TYPE)='HIGHWAYS'ANDTRIM(D.CODETEXT)='BASE'THEN11400010", "TRIM(F.DISTRICT_TYPE)='HEC'ANDTRIM(D.CODETEXT)='BASE'THEN11400020", "TRIM(F.DISTRICT_TYPE)='VILLAGE'ANDTRIM(D.CODETEXT)='BASE'THEN11400030", "TRIM(F.DISTRICT_TYPE)='GOVERNMENT'ANDTRIM(D.CODETEXT)='BASE'THEN11400040", "TRIM(F.DISTRICT_TYPE)='TOWNPLAN'ANDTRIM(D.CODETEXT)='BASE'THEN11400050", "TRIM(E.DISTRICT_NAME)='CRITICALITYINDEX(HEAVYTRAFFICBOUNDARY)'ANDTRIM(D.CODETEXT)='BASE'THEN11400060", "TRIM(E.DISTRICT_NAME)='CRITICALITYINDEX(LARGEPRIVATEESTATEORDEVELOPMENT)'ANDTRIM(D.CODETEXT)='BASE'THEN11400070", "TRIM(E.DISTRICT_NAME)='CRITICALITYINDEX(TEMPERATURESENSORCORRIDOR)'ANDTRIM(D.CODETEXT)='BASE'THEN11400080"]


In [8]:
logics = []

spl_chr = "THEN"
for txt in txts:
    if txt != "":
        logic = {}
        logic[txt.rpartition(spl_chr)[2]] = txt.rpartition(spl_chr)[0]
        logics.append(logic)
    
print(logics)

[{'11400010': "TRIM(F.DISTRICT_TYPE)='HIGHWAYS'ANDTRIM(D.CODETEXT)='BASE'"}, {'11400020': "TRIM(F.DISTRICT_TYPE)='HEC'ANDTRIM(D.CODETEXT)='BASE'"}, {'11400030': "TRIM(F.DISTRICT_TYPE)='VILLAGE'ANDTRIM(D.CODETEXT)='BASE'"}, {'11400040': "TRIM(F.DISTRICT_TYPE)='GOVERNMENT'ANDTRIM(D.CODETEXT)='BASE'"}, {'11400050': "TRIM(F.DISTRICT_TYPE)='TOWNPLAN'ANDTRIM(D.CODETEXT)='BASE'"}, {'11400060': "TRIM(E.DISTRICT_NAME)='CRITICALITYINDEX(HEAVYTRAFFICBOUNDARY)'ANDTRIM(D.CODETEXT)='BASE'"}, {'11400070': "TRIM(E.DISTRICT_NAME)='CRITICALITYINDEX(LARGEPRIVATEESTATEORDEVELOPMENT)'ANDTRIM(D.CODETEXT)='BASE'"}, {'11400080': "TRIM(E.DISTRICT_NAME)='CRITICALITYINDEX(TEMPERATURESENSORCORRIDOR)'ANDTRIM(D.CODETEXT)='BASE'"}]


In [39]:
check = list(logics[0].keys())[0]
print(check)
for table in tablelist:
    sqlstyle = "SELECT * FROM {table} WHERE g3e_sno = '{styleid}'".format(table = table, styleid = check)
    print(sqlstyle)
    try:
        cursor.execute(sqlstyle)
        fetch = cursor.fetchall()
        print(fetch)
        styletable = table
    except Exception as e:
        print(e)      


11400010
SELECT * FROM G3E_STYLETYPE_OPTLANG WHERE g3e_sno = '11400010'
ORA-00904: "G3E_SNO": invalid identifier
SELECT * FROM G3E_STYLEOBJECTS_OPTLANG WHERE g3e_sno = '11400010'
[(4351, '0009', 11400010, 'ADM_GPB_SHP_1_1', 'AreaStyle', b'(\xaa\x896\x0f\x1aOI\x90\x05\xe3r\xc4\xd0\x9d\xbd\x06\x00\x00\x00\x01\x00\x00\x00\xfd\xff\xff\xff\xfd\xff\xff\xff\x00\x00\x00\x00\x00\x00\x14@\x9a\x99\x99\x99\x99\x99\xc9?\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01!\xecmg\xb4}\xd3\x11\x90\xcc\x08\x0065\x19\x03\x02\x00\x00\x00\x01\x00\xff\xa0\x00\x00\xc7\xfbq\xfb\xe5\x93\xf6?\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00Q\xabL\xb3\x02\x80\xd3\x11\x90\xcc\x08\x0065\x19\x03\x02\x00\x00\x00\x85\x96\x98\x00\x01\x00\x00\x00\x01\x04\x00\x00\x00\x0c\x93\xa9\x82Quj@cb\xf3qm\xfcM@\n\xdc\xba\x9b\xa7*%@cb\xf3qm\xfcM@\x00\x00\x00\x00\x08\x00\x00\x00B\x00O\x00T\x00H\x00\xd2\x06\xae\x00\x00\x00\x00\x00J\xf3\xad\x00d\x00\x00\x00d\x00\x00\x00', 0, 1, datetime.datetime(2020, 3, 20, 18, 6, 52))]
SELECT * FROM G3E_STY

dict_keys(['11400010'])


In [5]:
# close connection
if connection:
    cursor.close()
    connection.close()
    print("Connection closed")

Connection closed
